In [ ]:
import numpy as np
import tensorflow as tf
learn = tf.contrib.learn
tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
mnist = learn.datasets.load_dataset('mnist')
data = mnist.train.images
labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

In [ ]:
feature_columns = learn.infer_real_valued_columns_from_input(data)
classifier = learn.LinearClassifier(feature_columns=feature_columns, n_classes=10)
classifier.fit(data, labels, batch_size=100, steps=1000)
classifier.evaluate(eval_data, eval_labels)

In [ ]:
def conv_model(x, is_training):
    # Reshape X to 4-D tensor: [batch, width, height, channels]
    x = tf.reshape(x, [-1, 28, 28, 1])
    # first conv layer will compute 32 features for each 5x5 patch
    h_conv1 = tf.contrib.layers.conv2d(x, num_outputs=32, kernel_size=[5, 5])
    h_pool1 = tf.contrib.layers.max_pool2d(h_conv1, [2, 2], padding='SAME')
    h_conv2 = tf.contrib.layers.conv2d(h_pool1, num_outputs=64,
                                       kernel_size=[5, 5])
    h_pool2 = tf.contrib.layers.max_pool2d(h_conv2, [2, 2], padding='SAME')
    # Reshape tensor into a batch of vectors
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    # Densely connected layer with 1024 neurons
    h_fcl = tf.contrib.layers.relu(h_pool2_flat, 1024)
    h_dropout = tf.contrib.layers.dropout(
        h_fcl, keep_prob=0.5, is_training=is_training)
    # Logits
    logits = tf.contrib.layers.linear(h_dropout, 10)
    return logits  

In [ ]:
def model_fn(x, y, mode)
    logits = conv_model(x, mode == tf.config.learn.ModeKeys.TRAIN)
    if mode != tf.contrib.learn.ModeKeys.INFER:
        loss = tf.contrib.losses.softmax_cross_entropy(logits, y)
        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            train_op = tf.contrib.layers.optimize_loss(loss,
                                                       tf.contrib.framework.get_global_step(),
                                                       0.001, 'SGD')
    return logits, loss, train_op

In [ ]:
classifier = learn.Classifier(model_fn=model_fn, n_classes=10)
classifier.fit(data, labels, batch_size=100, steps=20000)
classifier.evaluate(eval_data, eval_labels)